In [ ]:
# Import des packages 
import pandas as pd
import numpy as np
import sklearn
from scipy import sparse

import os 
import pathlib

# Insérez votre code ici
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay



In [ ]:
# On importe le dataset complet mais reduit en 100*100

path_to_data = r"C:\Users\Inrae\Documents\Projet_Data_Science"
file_path = os.path.join(path_to_data,"radio_tab.csv")

df = pd.read_csv(file_path)

df.shape


In [ ]:
# On cree un jeu de données réduit en respectant les proportions initiales 

df.rename(columns = {'Unnamed: 0':'FILENAME'}, inplace = True)

df = df.set_index("FILENAME")

#df = df.rename(columns = {'Unnamed : 0' : 'FILENAME'})
df.head()

In [ ]:
# On fait un X et Y 
# # On convertit en entier pour économiser en mémoire

X = df.iloc[:,5:].astype("int")

target = df["URL"] + "_"  + df["TYPE"]

type = df["TYPE"]

source = df["URL"]

X.head()



In [ ]:
X.dtypes
X.shape

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, type, stratify=target, test_size=0.3, random_state=42)

# On fait une ACP
## Pourcentage de variance expliquée par les axes


In [ ]:

pca = PCA(n_components=0.95)

pca.fit(X_train)

data_2D = pca.fit_transform(X_train)

import seaborn as sns 

# Avec les 10 premières composantes 

print("Avec les 10 premieres composantes, on explique : ", pca.explained_variance_ratio_[:11].sum().round(3) * 100 , " % de la variabilité")


print("Pour avoir 95 pour cent  de la variabilité initiale il faut : ", pca.explained_variance_ratio_.shape[0], "composantes")

In [ ]:
# Histogramme des parts de variance de l'ACP

sns.barplot(pca.explained_variance_ratio_[:11])
plt.title("Pourcentage de variance expliquée par les 10 premiers axes de l'ACP")


In [ ]:
# Idem en variance cumulée
sns.barplot(pca.explained_variance_ratio_.cumsum()[:21])
plt.title("Pourcentage de variance cumulée expliquée par les 20 premiers axes de l'ACP")


## Projection sur les axes 1 et 2 de l'ACP

In [ ]:
# On retrouve l'information source
X_train.head()
url = df["URL"]

source_train = X_train.iloc[:,1:2].merge(url, how="left", left_index = True, right_index = True).reset_index

source = source_train["URL"]

In [ ]:
# On visualise la source sur les deux premiers axes de l'ACP
sns.relplot(x=data_2D[:, 0], y=data_2D[:, 1], hue = source, cmap=plt.cm.Spectral)

print("La part de variance expliquée est", round(pca.explained_variance_ratio_.sum(),2))

abcisse = 'PCA 1, variance : ' +  str(pca.explained_variance_ratio_[0].round(3))
ordonnées = 'PCA 2, variance : ' +  str(pca.explained_variance_ratio_[1].round(3))
plt.xlabel(abcisse)
plt.ylabel(ordonnées)

plt.title("Données projetées sur les 2 axes de PCA" + " (" + str(round(pca.explained_variance_ratio_[:2].sum(),2)) +")")
plt.show();

In [ ]:
# On visualise la catégorie sur les deux premiers axes de l'ACP
type = y_train

sns.relplot(x=data_2D[:, 0], y=data_2D[:, 1], hue = type, cmap=plt.cm.Spectral)

abcisse = 'PCA 1, variance : ' +  str(pca.explained_variance_ratio_[0].round(3))
ordonnées = 'PCA 2, variance : ' +  str(pca.explained_variance_ratio_[1].round(3))
plt.xlabel(abcisse)
plt.ylabel(ordonnées)

plt.title("Données projetées sur les 2 axes de PCA : (" + str(round(pca.explained_variance_ratio_[:2].sum(),2)) +")")
plt.show();

In [ ]:
# On fait un graph par catégorie pour y voir plus clair

sns.relplot(x=data_2D[:, 0], y=data_2D[:, 1], hue = type, cmap=plt.cm.Spectral, col=type)

plt.show();

## Projection sur les axes 3 et 4 de l'ACP

In [ ]:
# Visualisation de l'information source sur les axes 3 et 4
sns.relplot(x=data_2D[:, 2], y=data_2D[:, 3], hue = source, cmap=plt.cm.Spectral)

abcisse = 'PCA 3, variance : ' +  str(pca.explained_variance_ratio_[2].round(3))
ordonnées = 'PCA 4, variance : ' +  str(pca.explained_variance_ratio_[3].round(3))
plt.xlabel(abcisse)
plt.ylabel(ordonnées)

plt.title("Données projetées sur les 2 axes de PCA" + " (" + str((pca.explained_variance_ratio_[2] + pca.explained_variance_ratio_[3]).round(3)) +")")
plt.show();

In [ ]:
# Visualiation des catégories sur les axes 3 et 4
sns.relplot(x=data_2D[:, 2], y=data_2D[:, 3], hue = type, cmap=plt.cm.Spectral)

abcisse = 'PCA 3, variance : ' +  str(pca.explained_variance_ratio_[2].round(3))
ordonnées = 'PCA 4, variance : ' +  str(pca.explained_variance_ratio_[3].round(3))
plt.xlabel(abcisse)
plt.ylabel(ordonnées)

plt.title("Données projetées sur les 2 axes de PCA" + " (" + str((pca.explained_variance_ratio_[2] + pca.explained_variance_ratio_[3]).round(3)) +")")
plt.show();

In [ ]:
# On fait un graphique par type pour mieux voir
sns.relplot(x=data_2D[:, 2], y=data_2D[:, 3], hue = type, cmap=plt.cm.Spectral, col=type)

plt.show();

## Visualisation des composantes

In [ ]:
# On visualise les 4 premières composantes de l'ACP

for i in range(10):
    plt.figure()
    plt.imshow(pca.components_[i].reshape(100,100), cmap = 'gray')


## Test d'un modèle de Random Forest à partir des résultats de l'ACP

In [ ]:
# Transformation des jeux de données

X_train_pca = pca.transform(X_train)

X_test_pca = pca.transform(X_test)

### Création d'un premier modèle

In [ ]:
# Application d'un premier modèle de Forêt aléatoire
from sklearn.ensemble import RandomForestClassifier

# On separe apprentissage et validation

rf = RandomForestClassifier()

rf.fit(X_train_pca, y_train)

# A voir en ajustant les parametres

y_pred = rf.predict(X_test_pca)

#rf.score(y_test, y_pred)



In [ ]:
print(sklearn.metrics.classification_report(y_test, y_pred))

### Optimisation du modèle

In [ ]:
# On cherche à optimiser les paramètres de la foret aléatoire

params = {
# Number of trees in random forest
"n_estimators" : [100, 200, 300, 400, 500, 600, 700, 800, 900]
}

crossval = KFold(n_splits = 3, random_state = 42, shuffle = True)

grid = GridSearchCV(estimator = rf, param_grid = params, cv=crossval, scoring = 'f1_macro')
grid.fit(X_train, y_train)
print(grid.best_params_)


# alpha_grid = {'alpha':[1/i for i in range(1,10)]}
# grid = GridSearchCV(estimator = lasso, param_grid = alpha_grid, cv=crossval, scoring = 'neg_mean_squared_error')
# grid.fit(X_train, y_train)
# print(grid.best_params_)


### Performances du nouveau modèle

In [ ]:
# On reapplique le meilleur  de rf
rf = RandomForestClassifier(n_estimators = 200)

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

In [ ]:
print(sklearn.metrics.classification_report(y_test, y_pred))

In [ ]:
# Matrice de confusion

cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["COVID", "Lung Opacity", "Normal", "Viral Pneumonia"])
disp.plot(cmap='Blues')

## Test de sélection des variables les plus pertinentes

In [ ]:
# On regarde si certaines zones sont plus variable que d'autre sur les images
# Ce n est pas applicable a du multilabel, on comparera juste COVID ou non

from sklearn.feature_selection import SelectPercentile

cible = np.where(type=="COVID",1,0)


# # On utilise variance threshold pour supprimer les variables qui ne varient pas 

# from sklearn.feature_selection import VarianceThreshold

# sel = VarianceThreshold(threshold=1e-6)

# X2 = sel.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, cible, test_size = .2, random_state = 84, shuffle = True)

sel = SelectPercentile(percentile = 90)
sel.fit(X_train, y_train)

plt.imshow(sel.get_support().reshape(299,299), cmap = 'gray_r');

plt.title("Masque en conservant les 90% de variables les plus importantes")

In [ ]:

sel = SelectPercentile(percentile = 80)
sel.fit(X_train, y_train)

plt.imshow(sel.get_support().reshape(299,299), cmap = 'gray_r');

plt.title("Masque en conservant les 80% de variables les plus importantes")

In [ ]:
sel = SelectPercentile(percentile = 70)
sel.fit(X_train, y_train)

plt.imshow(sel.get_support().reshape(299,299), cmap = 'gray_r');
plt.title("Masque en conservant les 70% de variables les plus importantes")